In [ ]:
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms
import numpy as np
import cv2
from pathlib import Path
from sklearn.metrics import f1_score
from typing import Tuple, Sequence, Optional

from models.Model_withResNet import DeepclfwithResNet

IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
VIDEO_EXTS = {".avi", ".mp4"}

In [ ]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### 얼굴 탐지 함수 선언
- input : Image.Image 타입의 single image
- output : target_size의 얼굴 중심으로 크롭된 Image.Image

In [ ]:
def detect_and_crop_face_optimized(
    image: Image.Image,
    target_size: Tuple[int, int] = (224, 224),
    providers: Sequence[str] = ("CUDAExecutionProvider", "CPUExecutionProvider"),
    model: str = "buffalo_s"
) -> Optional[Image.Image]:
    """
    입력: PIL.Image (임의 모드)
    출력: PIL.Image (RGB, 224x224)
    처리:
      1 얼굴 찾을 때만 원본을 긴 변 기준으로 512로 리사이즈해서 detect
      2 detect된 코 좌표를 원본 좌표로 되돌림 (scale 역산)
      3 원본에서 중심 기준 512x512 crop (작은 이미지는 256 리사이즈 후 사용)
      4 512→256으로 축소 (작은 이미지는 이미 256)
      5 256 가운데 224×224 center crop
      6 항상 RGB PIL.Image로 반환
    """

    if image is None:
        return None

    # (0) 모든 입력 이미지를 RGB로 통일
    if image.mode != "RGB":
        image = image.convert("RGB")

    # PIL → numpy (RGB → BGR)
    img_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    H, W = img_np.shape[:2]

    # (1) 얼굴 검출기 준비
    app = FaceAnalysis(name=model, providers=list(providers))
    app.prepare(ctx_id=0)

    # (2) 탐지를 위해 긴 변 기준 512로 축소
    detect_side = 512
    scale = detect_side / max(H, W)
    det_w = int(W * scale)
    det_h = int(H * scale)
    det_img = cv2.resize(img_np, (det_w, det_h))

    # (3) 얼굴 탐지
    faces = app.get(det_img)
    if not faces:
        cx, cy = W // 2, H // 4
    else:
        f = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
        cx, cy = f.kps[2]
        cx, cy = int(cx / scale), int(cy / scale)

    # (4) 크롭 크기 결정
    crop_size = 512 if (H >= 512 and W >= 512) else 256
    half = crop_size // 2
    x1 = min(max(cx - half, 0), max(W - crop_size, 0))
    y1 = min(max(cy - half, 0), max(H - crop_size, 0))
    cropped = img_np[y1:y1 + crop_size, x1:x1 + crop_size]
    if cropped is None or cropped.size == 0:
        return None

    # (5) 512일 경우 256으로 축소, 작은 이미지는 그대로 256
    resized_256 = cv2.resize(cropped, (256, 256), interpolation=cv2.INTER_AREA)

    # (6) 중앙 224×224 crop
    start = (256 - 224) // 2
    end = start + 224
    face_np = resized_256[start:end, start:end]

    # (7) numpy → PIL, 항상 RGB로 보장
    face_rgb = cv2.cvtColor(face_np, cv2.COLOR_BGR2RGB)
    face_img = Image.fromarray(face_rgb).convert("RGB")

    return face_img

In [ ]:
### 전처리한 이미지 저장 -> 학습시마다 전처리를 다시 수행할 필요 없음

def preprocess_and_save_dataset(
        input_root: str, 
        output_root: str, 
        center_frame_only: bool = True
):
    """
    특정 폴더 내의 이미지 및 비디오에서 얼굴을 검출/크롭하여 다른 폴더에 저장합니다.

    Args:
        input_root (str): 원본 데이터셋 루트 폴더 (real/fake 포함).
        output_root (str): 전처리된 이미지를 저장할 대상 루트 폴더.
        center_frame_only (bool): 비디오에서 중앙 프레임만 추출할지 여부.
                                  (False 시, 비디오 전체 프레임에서 얼굴 검출 시도)
    """
    input_root_path = Path(input_root)
    output_root_path = Path(output_root)
    
    # 폴더 구조 생성
    for label_name in ["real", "fake"]:
        (output_root_path / label_name).mkdir(parents=True, exist_ok=True)

    print(f"--- 데이터 전처리 시작 (Input: {input_root} -> Output: {output_root}) ---")
    
    total_files_processed = 0
    total_faces_saved = 0

    for label_name in ["real", "fake"]:
        input_dir = input_root_path / label_name
        output_dir = output_root_path / label_name
        
        if not input_dir.is_dir():
            print(f"경고: 입력 라벨 디렉토리 '{input_dir}'를 찾을 수 없습니다. 건너뜁니다.")
            continue
            
        print(f"\n[{label_name} 폴더] 처리 중...")

        for file_path in input_dir.iterdir():
            ext = file_path.suffix.lower()
            file_name_base = file_path.stem
            
            total_files_processed += 1
            faces_in_file = 0
            
            try:
                if ext in IMAGE_EXTS:
                    # 1. 이미지 파일 처리 (FaceData.__getitem__의 로직 차용)
                    image = Image.open(file_path).convert('RGB')
                    face_img = detect_and_crop_face_optimized(image)
                    
                    if face_img:
                        output_path = output_dir / f"{file_name_base}.{TARGET_IMAGE_FORMAT}"
                        face_img.save(output_path)
                        faces_in_file += 1

                elif ext in VIDEO_EXTS:
                    # 2. 비디오 파일 처리 (FaceData.__init__ 및 __getitem__ 로직 차용)
                    cap = cv2.VideoCapture(str(file_path))
                    if not cap.isOpened():
                        print(f"  경고: 비디오 파일을 열 수 없습니다: {file_path}")
                        continue
                        
                    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    
                    if total_frames > 0:
                        frame_indices = []
                        if center_frame_only:
                            # FaceData 클래스의 수정된 __init__ 로직
                            frame_indices.append(total_frames // 2) 
                        else:
                            # FaceData 클래스의 기존 __init__ 로직 (일정한 간격 추출)
                            frame_indices = np.linspace(
                                0, total_frames - 1, 30, dtype=int
                            ) 
                        
                        for i, idx in enumerate(frame_indices):
                            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                            ret, frame = cap.read()
                            
                            if not ret: continue
                                
                            # BGR (cv2) -> RGB (PIL) 변환
                            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                            face_img = detect_and_crop_face_optimized(image)
                            
                            if face_img:
                                # 비디오는 프레임 인덱스를 파일명에 추가하여 저장
                                output_path = output_dir / f"{file_name_base}_frame_{i:04d}.{TARGET_IMAGE_FORMAT}"
                                face_img.save(output_path)
                                faces_in_file += 1
                                
                    cap.release()
                
                if faces_in_file > 0:
                    total_faces_saved += faces_in_file
                    print(f"  ✅ {file_path.name} 처리 완료. {faces_in_file}개 저장.")
                # else:
                #     print(f"  ❌ {file_path.name}에서 얼굴을 찾지 못했습니다.")
                    
            except Exception as e:
                print(f"  ❌ 오류: 파일 {file_path} 처리 중 문제 발생: {e}")

    print("\n--- 전처리 요약 ---")
    print(f"총 스캔 파일 수: {total_files_processed}")
    print(f"총 저장된 얼굴 이미지 수: {total_faces_saved}")
    print(f"저장 폴더: {output_root}")

### 데이터를 저장할 경우 해당 함수 실행
# preprocess_and_save_dataset(input_root='data', output_root='preprocessed_data', center_frame_only=True)

### Dataset 클래스
- train set에서 불러오는 형태(이미지를 batch 단위로 반환 가능)
- transform이 None이면, 이미 전처리가 완료된 데이터를 받아서 처리
- transform이 None이 아니면, 원본 데이터를 받아서 transform에 전달된 전처리 함수를 통해서 처리

In [ ]:
# 데이터 파일은 처리의 용이성을 위해 real/fake로 하위 폴더를 만들어서 저장

### Dataset 기반의 클래스

class FaceData(Dataset):
    """
    얼굴 이미지/비디오 데이터셋 클래스.
    
    root 디렉토리 구조는 다음과 같다고 가정합니다:
    root/
        real/
            ... (이미지 및 비디오 파일)
        fake/
            ... (이미지 및 비디오 파일)
    """

    def __init__(
            self,
            root="data",
            num_frames_to_extract=30, # 비디오에서 추출할 프레임 수
            transform=None,
            is_preprocessed=False,
    ):
        self.root = Path(root)
        self.transform = transform
        self.num_frames_to_extract = num_frames_to_extract
        self.is_preprocessed = is_preprocessed
        
        # self.samples는 (file_path, frame_index, label) 튜플을 저장합니다.
        # frame_index가 None이면 이미지 파일을 의미합니다.
        self.samples = [] 
        
        # 폴더 이름(real/fake)에 따라 라벨링 (real=1, fake=0)
        label_map = {"real": 0, "fake": 1}

        if not self.root.is_dir():
            raise FileNotFoundError(f"루트 디렉토리를 찾을 수 없습니다: {root}")

        print("데이터셋 샘플을 스캔 중입니다...")
        for label_name, label_id in label_map.items():
            data_dir = self.root / label_name
            if not data_dir.is_dir():
                print(f"경고: 라벨 디렉토리를 찾을 수 없습니다: {data_dir}")
                continue
                
            for file_path in data_dir.iterdir():
                ext = file_path.suffix.lower()
                
                try:
                    if ext in IMAGE_EXTS:
                        # 이미지 파일: (파일경로, None, 라벨)
                        self.samples.append((file_path, None, label_id))
                        
                    elif ext in VIDEO_EXTS:
                        # 비디오 파일: 프레임 인덱스별로 샘플 추가
                        cap = cv2.VideoCapture(str(file_path))
                        if not cap.isOpened():
                            print(f"경고: 비디오 파일을 열 수 없습니다: {file_path}")
                            continue
                            
                        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                        
                        # total_frames가 0보다 큰 경우에만 처리
                        if total_frames > 0:
                            
                            # 👇 **[수정된 부분]** 👇
                            # 1. 중앙 프레임 인덱스 계산
                            # total_frames는 1부터 시작하는 개수, 인덱스는 0부터 시작
                            center_frame_index = total_frames // 2
                            
                            # 2. 중앙 프레임 하나만 self.samples에 추가
                            self.samples.append((file_path, center_frame_index, label_id))
                            # 👆 **[수정된 부분]** 👆

                        cap.release()
                    
                except Exception as e:
                    print(f"오류: 파일 {file_path} 처리 중 문제 발생: {e}")
                            
        if not self.samples:
            print(f"경고: {root} 디렉토리에서 처리할 샘플을 찾지 못했습니다.")
        else:
            print(f"총 {len(self.samples)}개의 샘플을 찾았습니다.")

    def __len__(self):
        """데이터셋의 총 샘플 수 (총 얼굴 이미지 수)를 반환합니다."""
        return len(self.samples)

    def __getitem__(self, index):
        """
        인덱스에 해당하는 단일 샘플(얼굴 이미지와 라벨)을 반환합니다.
        """
        file_path, frame_index, label = self.samples[index]
        
        face_img = None
        
        try:
            if frame_index is None: # Case 1: 이미지 파일
                # Image.open은 RGB 순서로 이미지를 엽니다.
                image = Image.open(file_path).convert('RGB')
                if self.is_preprocessed is False :
                    face_img = detect_and_crop_face_optimized(image)
                else:
                    face_img = image
            
            else: # Case 2: 비디오 프레임
                # cv2.VideoCapture는 스레드 안전하지 않을 수 있으므로,
                # num_workers > 0 일 때를 대비해 __getitem__ 내에서 매번 엽니다.
                cap = cv2.VideoCapture(str(file_path))
                if not cap.isOpened():
                     raise IOError(f"비디오 파일을 열 수 없습니다: {file_path}")
                
                # 지정된 프레임 인덱스로 이동
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
                ret, frame = cap.read() # frame은 BGR 순서
                cap.release()
                
                if not ret:
                    raise IOError(f"프레임 {frame_index}을 읽는 데 실패했습니다: {file_path}")
                    
                # BGR (cv2) -> RGB (PIL) 변환
                image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                if self.is_preprocessed is False:
                    face_img = detect_and_crop_face_optimized(image)
                else:
                    face_img = image

            # 얼굴 검출 실패 시 (face_img가 None이거나 비어있을 경우)
            if face_img is None:
                # print(f"경고: {file_path} (프레임: {frame_index})에서 얼굴 검출 실패. 검은색 이미지 반환.")
                # 검은색 빈 이미지를 생성하여 학습은 가능하도록 함
                face_img = Image.new('RGB', (224, 224), (0, 0, 0)) 
                
            # 공통 전처리 (Transform) 적용 (e.g., ToTensor, Normalize)
            if self.transform:
                face_img = self.transform(face_img)
                
            return face_img, label
        
        except Exception as e:
            # 데이터 로딩 중 오류 발생 시, DataLoader가 멈추는 것을 방지
            print(f"오류: 인덱스 {index} ({file_path}, 프레임 {frame_index}) 로딩 중 문제 발생: {e}")
            
            # 오류 발생 시 대체할 더미 데이터 반환
            dummy_img = Image.new('RGB', (224, 224), (0, 0, 0)) # 224x224 크기 가정
            if self.transform:
                dummy_img = self.transform(dummy_img)
            
            # -1과 같은 특수 라벨을 반환하여 나중에 배치에서 제외할 수 있도록 함
            return dummy_img, -1

In [ ]:
from torch.utils.data import random_split

NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=NORMALIZE_MEAN, std=NORMALIZE_STD),
])

datasets = FaceData(transform=transform, root="data", is_preprocessed=False)

# 이미 크롭된 데이터가 저장되어 있다면 해당 함수 실행
# datasets = FaceData(transform=transform, root="preprocessed_data", is_preprocessed=True)

total_size = len(datasets)
train_size = int(total_size * 0.8)
val_size = total_size - train_size

# 2. random_split으로 데이터셋 분리
train_dataset, val_dataset = random_split(datasets, [train_size, val_size])

# 3. DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False) # Val은 섞을 필요 없음

In [ ]:
# 스케줄러에 관한 하이퍼 파라미터
STEP_SIZE=10
GAMMA=0.1

# 모델 학습에 관한 하이퍼 파라미터
NUM_EPOCHS=20
NUM_CLASSES=2
UNFREEZE_EPOCH=10


In [ ]:
model = DeepclfwithResNet(num_classes=NUM_CLASSES).to(device)

In [ ]:
for param in model.rgb_branch.parameters():
    param.requires_grad = False

trainable_params = filter(lambda p : p.requires_grad, model.parameters())

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(trainable_params, lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

num_epochs = NUM_EPOCHS
is_unforzen = False

for epoch in range(num_epochs):
    if not is_unfrozen and epoch == UNFREEZE_EPOCH:
        print(f"\n--- Epoch {epoch + 1}: Unfreezing 'rgb_branch' parameters! ---")
        
        for param in model.rgb_branch.parameters():
            param.requires_grad = True
            
        optimizer.add_param_group({
            'params': model.rgb_branch.parameters(), 
            'lr': 0.0001 # 예: 기존 LR(0.001)보다 낮은 값
        })
        
        is_unfrozen = True # 플래그 업데이트

    model.train()
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 50 == 49:
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 100:.3f}")
            running_loss = 0.0


    model.eval() # 모델을 평가 모드로 설정 (Dropout, BatchNorm 비활성화)
    
    all_preds = []
    all_labels = []

    with torch.no_grad(): # 기울기 계산을 중지하여 메모리 절약 및 속도 향상
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

            # 모델 예측
            val_outputs = model(val_inputs)
            
            # 가장 높은 확률을 가진 클래스를 예측값으로 선택
            # val_outputs shape: (Batch_Size, Num_Classes)
            preds = torch.argmax(val_outputs, dim=1) 
            
            # sklearn 계산을 위해 CPU로 이동 후 numpy 배열로 변환
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(val_labels.cpu().numpy())

    # --- 3. Macro F1 스코어 계산 ---
    # 수집된 모든 예측값과 실제 라벨을 사용하여 F1 스코어 계산
    # zero_division=0 : 특정 클래스에 대한 예측이 없어 분모가 0이 될 경우 0.0으로 처리
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    
    print(f"--- [Epoch {epoch + 1}] Validation Macro F1 Score: {macro_f1:.4f} ---")

    scheduler.step()
    
    # --- 4. 모델 저장 ---
    # (기존 코드)
    if epoch % 5 == 4:
        torch.save(model.state_dict(), f"CNN_Deepfake_Classifier_{epoch}.pth")

print("Finish Training")